In [1]:
import os
import logging
from datetime import datetime

In [2]:


import boto3
import sagemaker
from sagemaker.session import TrainingInput
from sagemaker import image_uris
from sagemaker import hyperparameters



sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\hemch\AppData\Local\sagemaker\sagemaker\config.yaml


In [3]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)

In [4]:
region = sagemaker.Session().boto_region_name
print(region)

us-east-1


In [5]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import json
import logging
import sys
from pathlib import Path

import ipytest

#CODE_FOLDER = Path("code")
#sys.path.extend([f"./{CODE_FOLDER}"])

#DATA_FILEPATH = "../data/penguins.csv"

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

In [6]:
import os

bucket = os.environ["BUCKET"]+'/sagemaker-course'
role = os.environ["ROLE"]

COMET_API_KEY = os.environ.get("COMET_API_KEY", None)
COMET_PROJECT_NAME = os.environ.get("COMET_PROJECT_NAME", None)

In [7]:
bucket

'beercafe-ml-bucket/sagemaker-course'

In [8]:
prefix = "iris"

In [9]:
!aws s3 ls {bucket}/{prefix}/

                           PRE batch_transform/
                           PRE data/
                           PRE model/


In [10]:
!aws s3 ls {bucket}/{prefix}/data/ --recursive

2024-04-16 21:40:09        950 sagemaker-course/iris/data/iris_test.csv
2024-04-16 21:40:00       1900 sagemaker-course/iris/data/iris_train.csv


# Training

In [11]:
train_file = "data/iris_train.csv"
valid_file = "data/iris_test.csv"

train_file_uri = "s3://{}/{}/{}".format(bucket, prefix, train_file)
valid_file_uri = "s3://{}/{}/{}".format(bucket, prefix, valid_file)
print("train file uri:", train_file_uri)
print("valid file uri:", valid_file_uri)

train file uri: s3://beercafe-ml-bucket/sagemaker-course/iris/data/iris_train.csv
valid file uri: s3://beercafe-ml-bucket/sagemaker-course/iris/data/iris_test.csv


In [12]:
train_ip = TrainingInput(train_file_uri, content_type="csv")
valid_ip = TrainingInput(valid_file_uri, content_type="csv")
print(train_ip)
print(valid_ip)

In [13]:
model_op = "s3://{}/{}/{}".format(bucket, prefix, "model")
print(model_op)

s3://beercafe-ml-bucket/sagemaker-course/iris/model


In [14]:
train_image_uri = sagemaker.image_uris.retrieve("xgboost", region, "latest")
print(train_image_uri)

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [15]:
base_job_name = "iris-xgboost-"

In [21]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=role,
    base_job_name=base_job_name,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=model_op,
    sagemaker_session=sagemaker.Session(),
    volume_size=5
)

In [22]:
xgb_model.set_hyperparameters(
    num_class=3, max_depth=5, num_round=10, objective="multi:softmax",
)

In [23]:
job_name = base_job_name + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(job_name)

iris-xgboost-2024-04-16-22-38-06


In [24]:
xgb_model.fit({"train": train_ip, "validation": valid_ip}, wait=True, job_name=job_name)

INFO:sagemaker:Creating training-job with name: iris-xgboost-2024-04-16-22-38-06


2024-04-16 17:08:09 Starting - Starting the training job...
2024-04-16 17:08:24 Starting - Preparing the instances for training...
2024-04-16 17:09:03 Downloading - Downloading input data...
2024-04-16 17:09:43 Downloading - Downloading the training image......
2024-04-16 17:10:49 Training - Training image download completed. Training in progress.
2024-04-16 17:10:49 Uploading - Uploading generated training modelArguments: train
[2024-04-16:17:10:41:INFO] Running standalone xgboost training.
[2024-04-16:17:10:41:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8499.49mb
[2024-04-16:17:10:41:INFO] Determined delimiter of CSV input is ','
[17:10:41] S3DistributionType set as FullyReplicated
[17:10:41] 100x4 matrix with 400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2024-04-16:17:10:41:INFO] Determined delimiter of CSV input is ','
[17:10:41] S3DistributionType set as FullyReplicated
[17:10:41] 50x4 matrix

In [19]:
!aws s3 ls {bucket}/{prefix}/model/{job_name}/

# Inference

In [20]:
from sagemaker.serializers import CSVSerializer

## Deploy model as endpoint

In [25]:
type(xgb_model)

sagemaker.estimator.Estimator

In [27]:
xgb_predictor = xgb_model.deploy(initial_instance_count=1, instance_type="ml.t2.medium", serializer=CSVSerializer())

INFO:sagemaker:Creating model with name: iris-xgboost--2024-04-16-17-13-08-491
INFO:sagemaker:Creating endpoint-config with name iris-xgboost--2024-04-16-17-13-08-491
INFO:sagemaker:Creating endpoint with name iris-xgboost--2024-04-16-17-13-08-491


-------!

Predict single record

In [28]:
xgb_predictor.predict("7.7, 3.0, 6.1, 2.3")

b'2.0'

Endpoint

In [29]:
endpoint_name = xgb_predictor.endpoint_name
endpoint_name

'iris-xgboost--2024-04-16-17-13-08-491'

In [30]:
sagemaker_runtime = boto3.client("runtime.sagemaker")

Endpoint One record

In [31]:
payload_csv_text = "7.8, 3.0, 6.1, 2.3"
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response)

{'ResponseMetadata': {'RequestId': '189a1e48-33cf-49d1-97e5-208afd4b9d1b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '189a1e48-33cf-49d1-97e5-208afd4b9d1b', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Tue, 16 Apr 2024 17:30:55 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '3', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x000002BA968FA1A0>}


In [32]:
print(response["Body"].read().decode())

2.0


Multiple records

In [34]:
payload_csv_text = "7.7, 3.0, 6.1, 2.3 \n 7.9, 3.8, 6.4, 2.1"

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

2.0,2.0


Multiple records from a local file

In [36]:
csv_buffer = open("data/iris_infer.csv")
payload_csv_text = csv_buffer.read()

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

0.0,1.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,2.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,2.0,1.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,1.0,0.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,2.0,1.0,1.0


In [37]:
payload_csv_text

'4.4,3.2,1.3,0.2\n5.8,2.6,4.0,1.2\n5.4,3.4,1.5,0.4\n7.9,3.8,6.4,2.0\n7.0,3.2,4.7,1.4\n4.9,3.6,1.4,0.1\n7.7,2.8,6.7,2.0\n6.0,3.0,4.8,1.8\n4.7,3.2,1.3,0.2\n5.9,3.0,4.2,1.5\n5.4,3.7,1.5,0.2\n6.3,2.7,4.9,1.8\n5.1,2.5,3.0,1.1\n6.5,3.0,5.5,1.8\n6.7,3.3,5.7,2.1\n5.0,2.3,3.3,1.0\n6.1,2.9,4.7,1.4\n4.8,3.4,1.6,0.2\n4.6,3.2,1.4,0.2\n6.4,2.7,5.3,1.9\n4.9,2.5,4.5,1.7\n6.4,2.8,5.6,2.1\n5.9,3.0,5.1,1.8\n5.5,3.5,1.3,0.2\n5.2,3.4,1.4,0.2\n6.7,3.0,5.2,2.3\n6.3,2.5,4.9,1.5\n4.8,3.1,1.6,0.2\n6.0,2.2,5.0,1.5\n6.4,3.2,4.5,1.5\n6.8,3.0,5.5,2.1\n4.9,3.0,1.4,0.2\n4.5,2.3,1.3,0.3\n5.0,3.3,1.4,0.2\n6.0,2.7,5.1,1.6\n6.7,3.0,5.0,1.7\n5.2,4.1,1.5,0.1\n6.4,3.1,5.5,1.8\n6.6,2.9,4.6,1.3\n5.1,3.5,1.4,0.2\n5.7,2.6,3.5,1.0\n5.7,2.5,5.0,2.0\n5.5,2.5,4.0,1.3\n5.5,2.6,4.4,1.2\n6.5,3.2,5.1,2.0\n5.2,3.5,1.5,0.2\n5.0,3.2,1.2,0.2\n7.3,2.9,6.3,1.8\n6.0,2.2,4.0,1.0\n4.9,2.4,3.3,1.0\n'

Multiple records from a S3 file

In [42]:
import pandas as pd
import io
infer_ip_s3_uri = "s3://{}/{}/{}".format(
    bucket, prefix, "batch_transform/iris_infer.csv"
)


s3_clnt = boto3.client("s3")
obj = s3_clnt.get_object(Bucket='beercafe-ml-bucket', Key="sagemaker-course/iris/batch_transform/iris_infer.csv")
payload_df = pd.read_csv(obj["Body"])

csv_buffer = io.StringIO()
payload_df.to_csv(csv_buffer, header=None, index=None)
payload_csv_text = csv_buffer.getvalue()

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload_csv_text
)
print(response["Body"].read().decode())

1.0,0.0,2.0,1.0,0.0,2.0,2.0,0.0,1.0,0.0,2.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,2.0,1.0,2.0,2.0,0.0,0.0,2.0,1.0,0.0,2.0,1.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,1.0,0.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,2.0,1.0,1.0


## Very Imp- DELETE the endpoint so save cost

In [43]:
sagemaker_client = boto3.client("sagemaker")

In [45]:
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '0515ba1e-3535-4823-bc54-34e403c8a782',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0515ba1e-3535-4823-bc54-34e403c8a782',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 16 Apr 2024 17:37:51 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

## Batch Transform

In [46]:
batch_ip = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform")
batch_op = "s3://{}/{}/{}".format(bucket, prefix, "batch_transform_op")

print("batch_ip",batch_ip)
print("batch_op",batch_op)

batch_ip s3://beercafe-ml-bucket/sagemaker-course/iris/batch_transform
batch_op s3://beercafe-ml-bucket/sagemaker-course/iris/batch_transform_op


In [50]:
transformer = xgb_model.transformer(
    instance_count=1, instance_type="ml.m5.xlarge", output_path=batch_op
)

INFO:sagemaker:Creating model with name: iris-xgboost--2024-04-16-17-41-36-269


In [51]:
transformer.transform(
    data=batch_ip, data_type="S3Prefix", content_type="text/csv", split_type="Line"
)
transformer.wait()

INFO:sagemaker:Creating transform job with name: iris-xgboost--2024-04-16-17-41-48-191


.........................Arguments: serve
[2024-04-16 17:46:28 +0000] [1] [INFO] Starting gunicorn 19.9.0
[2024-04-16 17:46:28 +0000] [1] [INFO] Listening at: http://0.0.0.0:8080 (1)
[2024-04-16 17:46:28 +0000] [1] [INFO] Using worker: gevent
[2024-04-16 17:46:28 +0000] [21] [INFO] Booting worker with pid: 21
[2024-04-16 17:46:28 +0000] [22] [INFO] Booting worker with pid: 22
[2024-04-16 17:46:28 +0000] [23] [INFO] Booting worker with pid: 23
[2024-04-16 17:46:28 +0000] [24] [INFO] Booting worker with pid: 24
/opt/amazon/lib/python3.7/site-packages/gunicorn/workers/ggevent.py:65: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util.ssl_ (/opt/amazon/lib/python3.7/site-packages/urllib3/util/ssl_.py)', 'u